In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

'c:\\Users\\jaesc2\\GitHub\\skforecast'

In [1]:
import platform
import psutil
import skforecast
import pandas as pd
import numpy as np
import scipy
import sklearn
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMRegressor
from skforecast.recursive import ForecasterRecursiveMultiSeries
from skforecast.model_selection import grid_search_forecaster_multiseries
from skforecast.utils.utils import align_series_and_exog_multiseries
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import make_column_transformer
from skforecast.preprocessing import series_long_to_dict
from skforecast.preprocessing import exog_long_to_dict
from skforecast.datasets import fetch_dataset
from skforecast.utils import check_preprocess_series


%load_ext pyinstrument
%load_ext line_profiler

In [3]:
n = 5000
n_series = 500
series = pd.DataFrame(
    {f"series_{i}": np.random.randint(1, 100, n) for i in range(1, n_series)
    },
    index=pd.date_range(start="2023-01-01", periods=n, freq="D"),
)

freq = series.index.freq
series.index.name = "datetime"
series = series.reset_index()
series = pd.melt(series, id_vars="datetime", var_name="series_id", value_name="value")
series = series.groupby("series_id", sort=False).apply(
    lambda x: x.set_index("datetime").asfreq("D"), include_groups=False
)
display(series)

exog = pd.DataFrame(
    {
        "exog_1": np.random.randint(1, 10, n),
        "exog_2": np.random.randint(1, 10, n),
        "exog_3": np.random.randint(1, 10, n),
        "exog_4": np.random.randint(1, 10, n),
        "exog_5": np.random.randint(1, 10, n),
        "exog_6": np.random.randint(1, 10, n),
    },
    index=pd.date_range(start="2023-01-01", periods=n, freq="D"),
)
exog.index.name= "datetime"
exog = [exog.assign(series_id=f"series_{i}") for i in range(1, n_series)]
# exog = [exog_i.copy().sample(frac=0.8).sort_index() for exog_i in exog]
exog = pd.concat(exog)
exog = exog.set_index(["series_id", exog.index])
display(exog)

# Formato dict
series_dict = {
    sid: series.loc[sid]['value'] for sid in series.index.levels[0]
}

exog_dict = {
    sid: exog.loc[sid]for sid in exog.index.levels[0]
}


value
series_id  datetime         
series_1   2023-01-01     99
           2023-01-02     61
           2023-01-03     47
           2023-01-04     58
           2023-01-05     78
...                      ...
series_499 2036-09-04     92
           2036-09-05     70
           2036-09-06     60
           2036-09-07     22
           2036-09-08     89

[2495000 rows x 1 columns]

exog_1  exog_2  exog_3  exog_4  exog_5  exog_6
series_id  datetime                                                  
series_1   2023-01-01       1       6       9       8       1       1
           2023-01-02       5       7       2       8       7       8
           2023-01-03       7       7       1       7       3       9
           2023-01-04       1       9       5       2       3       3
           2023-01-05       8       7       8       2       6       3
...                       ...     ...     ...     ...     ...     ...
series_499 2036-09-04       8       6       7       1       2       8
           2036-09-05       9       4       1       8       4       9
           2036-09-06       6       4       4       3       6       9
           2036-09-07       2       9       1       4       8       6
           2036-09-08       7       9       5       9       5       5

[2495000 rows x 6 columns]

In [10]:
%%timeit

series_dict = {
    sid: series.loc[sid]['value'] for sid in series.index.levels[0]
}

116 ms ± 4.67 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
series_no_idx = series.reset_index()

In [13]:
%%timeit

series_2 = series_long_to_dict(series_no_idx, series_id="series_id", index="datetime", values="value", freq='D')

761 ms ± 15.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [47]:
exog = pd.DataFrame(
    {
        "exog_1": np.random.randint(1, 10, n),
        "exog_2": np.random.randint(1, 10, n),
        "exog_3": np.random.randint(1, 10, n),
        "exog_4": np.random.randint(1, 10, n),
        "exog_5": np.random.randint(1, 10, n),
        "exog_6": np.random.randint(1, 10, n),
    },
    index=pd.date_range(start="2023-01-01", periods=n, freq="D"),
)
exog.index.name= "datetime"

In [49]:
from skforecast.utils import transform_dataframe
from sklearn.preprocessing import StandardScaler

exog_tmp = transform_dataframe(
            df                = exog[["exog_4", "exog_5", "exog_6"]],
            transformer       = StandardScaler(),
            fit               = True,
            inverse_transform = False
        )
exog_tmp

,exog_4,exog_5,exog_6
datetime,,,
2023-01-01,-0.912871,-0.882523,-1.289317
2023-01-02,-0.912871,1.078639,-0.886405
2023-01-03,0.000000,1.078639,1.128152
2023-01-04,1.825742,-1.372813,1.128152
2023-01-05,0.000000,0.098058,-0.080582


In [ ]:
if exog_tmp.columns == exog_names_in_:
    exog[["exog_4", "exog_5", "exog_6"]] = exog_tmp
else:
    exog = pd.concat([exog, exog_tmp], axis=1)
    exog = exog.drop(columns=[col for col in exog.columns if col not in ["exog_4", "exog_5", "exog_6"]])

In [5]:
forecaster = ForecasterRecursiveMultiSeries(regressor=LGBMRegressor(verbose=-1), lags=5, transformer_exog=StandardScaler())
forecaster.transformer_series_ = {k: StandardScaler() for k in series.index.get_level_values(0).unique()}
forecaster.differentiator_ = {k: None for k in series.index.get_level_values(0).unique()}

In [49]:
_ = forecaster._create_train_X_y_v2(
    series = series,
    exog = exog
)

In [38]:
_2 = forecaster._create_train_X_y(
    series = series,
    exog = exog
)

In [6]:
import warnings
from skforecast.exceptions import MissingValuesWarning

warnings.simplefilter('ignore', category=MissingValuesWarning)  

In [10]:
%%timeit -n 5 -r 5

forecaster._create_train_X_y(
    series = series,
    exog = exog
)

491 ms ± 34.7 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


In [11]:
%%timeit -n 5 -r 5

forecaster._create_train_X_y_v2(
    series = series,
    exog = exog
)

562 ms ± 49.7 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


In [27]:
series.loc['series_1'].to_numpy().ravel()

array([95, 16, 60, ..., 15, 84, 40], shape=(5000,), dtype=int32)

In [ ]:
series.loc['series_1'].to_numpy().ravel()

In [20]:
forecaster._create_train_X_y_v2(
    series = series,
    exog = exog
)

TypeError: 'str' object is not callable

In [26]:
%%timeit -n 5 -r 5

forecaster._create_train_X_y(
    series = series_dict,
    exog = exog_dict
)

374 ms ± 22.5 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


In [20]:
forecaster.index_type_

pandas.core.indexes.datetimes.DatetimeIndex

In [16]:
type(series.index.levels[1])

pandas.core.indexes.datetimes.DatetimeIndex

In [18]:
[type(idx) for idx in series.index.levels]

[pandas.core.indexes.base.Index, pandas.core.indexes.datetimes.DatetimeIndex]

In [11]:
import pandas as pd

idx = pd.date_range('2024-01-01', periods=3, freq='D')

# Opción 1: Comparar como string
print(idx.dtype == 'datetime64[ns]')   # True

# Opción 2: Comparar usando numpy dtype
import numpy as np
print(idx.dtype == np.dtype('datetime64[ns]'))  # True

# Opción 3: Comprobar tipo de columna en DataFrame
df = pd.DataFrame(index=idx)
print(df.index.dtype == 'datetime64[ns]')  # True


True
True
True


In [12]:
np.dtype('datetime64[ns]')

dtype('<M8[ns]')

In [12]:
# %%timeit -r 5 -n 5
# _ = forecaster._create_train_X_y(
#     series = series,
#     exog=exog
# )

In [13]:
%%timeit -r 5 -n 5
_ = forecaster._create_train_X_y_v2(
    series = series,
    exog=exog
)

319 ms ± 9.64 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


In [20]:
_ = forecaster._create_train_X_y_v2(
                            series = series,
                            exog=exog
                        )

value


KeyError: 'value'

In [24]:
series_level_0 = series.index.get_level_values(0)
unique_series = series_level_0.unique()

# Opcional: crea un buffer de posiciones de inicio/fin para cada serie
idx_start = series_level_0.searchsorted(unique_series, side='left')
idx_start

array([     0,   5000,  10000,  15000,  20000,  25000,  30000,  35000,
        40000,  45000,  50000,  55000,  60000,  65000,  70000,  75000,
        80000,  85000,  90000,  95000, 100000, 105000, 110000, 115000,
       120000, 125000, 130000, 135000, 140000, 145000, 150000, 155000,
       160000, 165000, 170000, 175000, 180000, 185000, 190000, 195000,
       200000, 205000, 210000, 215000, 220000, 225000, 230000, 235000,
       240000, 245000, 250000, 255000, 260000, 265000, 270000, 275000,
       280000, 285000, 290000, 295000, 300000, 305000, 310000, 315000,
       320000, 325000, 330000, 335000, 340000, 345000, 350000, 355000,
       360000, 365000, 370000, 375000, 380000, 385000, 390000, 395000,
       400000, 405000, 410000, 415000, 420000, 425000, 430000, 435000,
       440000, 445000, 450000, 455000, 460000, 465000, 470000, 475000,
       480000, 485000, 490000])

In [6]:
def funt_to_profile(forecaster, series, exog):
    _ = forecaster._create_train_X_y_v2(
                            series = series,
                            exog=exog
                        )

%lprun -f forecaster._create_train_X_y_v2 funt_to_profile(forecaster, series, exog)

Timer unit: 1e-07 s

Total time: 1.19015 s
File: c:\Users\jaesc2\GitHub\skforecast\skforecast\recursive\_forecaster_recursive_multiseries.py
Function: _create_train_X_y_v2 at line 1426

Line #      Hits         Time  Per Hit   % Time  Line Contents
  1426                                               def _create_train_X_y_v2(
  1427                                                   self,
  1428                                                   series: pd.DataFrame,
  1429                                                   exog: pd.Series | pd.DataFrame | None = None,
  1430                                                   store_last_window: bool | list[str] = True,
  1431                                               ) -> tuple[
  1432                                                   pd.DataFrame,
  1433                                                   pd.Series,
  1434                                                   dict[str, pd.Index],
  1435                                      

In [16]:
%%timeit -r 5 -n 5
_ = forecaster._create_train_X_y_v3(
    series = series,
    exog=exog
)

493 ms ± 46.1 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


In [9]:
def funt_to_profile(forecaster, series, exog):
    _ = forecaster._create_train_X_y(
                            series = series,
                            exog=exog
                        )

%lprun -f forecaster._create_train_X_y funt_to_profile(forecaster, series, exog)

Timer unit: 1e-07 s

Total time: 0.98568 s
File: c:\Users\jaesc2\GitHub\skforecast\skforecast\recursive\_forecaster_recursive_multiseries.py
Function: _create_train_X_y at line 959

Line #      Hits         Time  Per Hit   % Time  Line Contents
   959                                               def _create_train_X_y(
   960                                                   self,
   961                                                   series: pd.DataFrame,
   962                                                   exog: pd.Series | pd.DataFrame | None = None,
   963                                                   store_last_window: bool | list[str] = True,
   964                                               ) -> tuple[
   965                                                   pd.DataFrame,
   966                                                   pd.Series,
   967                                                   dict[str, pd.Index],
   968                                             

In [20]:
%%timeit -r 5 -n 5
{
    sid: series.loc[sid]['value'] for sid in series.index.levels[0]
}

{
    sid: exog.loc[sid]for sid in exog.index.levels[0]
}

69.4 ms ± 5.9 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


In [21]:
%%timeit -r 5 -n 5
_ = forecaster._create_train_X_y(
    series = series_dict,
    exog=exog_dict
)

413 ms ± 44.8 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


In [22]:
def funt_to_profile(forecaster, series_dict, exog_dict):
    _ = forecaster._create_train_X_y(
                            series = series_dict,
                            exog=exog_dict
                        )

%lprun -f forecaster._create_train_X_y funt_to_profile(forecaster, series_dict, exog_dict)

Timer unit: 1e-07 s

Total time: 0.701712 s
File: c:\Users\jaesc2\Miniconda3\envs\skforecast_py11_2\Lib\site-packages\skforecast\recursive\_forecaster_recursive_multiseries.py
Function: _create_train_X_y at line 1087

Line #      Hits         Time  Per Hit   % Time  Line Contents
  1087                                               def _create_train_X_y(
  1088                                                   self,
  1089                                                   series: pd.DataFrame | dict[str, pd.Series | pd.DataFrame],
  1090                                                   exog: pd.Series | pd.DataFrame | dict[str, pd.Series | pd.DataFrame] | None = None,
  1091                                                   store_last_window: bool | list[str] = True,
  1092                                               ) -> tuple[
  1093                                                   pd.DataFrame,
  1094                                                   pd.Series,
  1095           

In [4]:
def funt_to_profile(series):
    check_preprocess_series(series=series)

%lprun -f check_preprocess_series funt_to_profile(series)

Timer unit: 1e-07 s

Total time: 0.064929 s
File: c:\Users\jaesc2\GitHub\skforecast\skforecast\utils\utils.py
Function: check_preprocess_series at line 2448

Line #      Hits         Time  Per Hit   % Time  Line Contents
  2448                                           def check_preprocess_series(
  2449                                               series: pd.DataFrame,
  2450                                           ) -> tuple[pd.DataFrame]:
  2451                                               """
  2452                                               Check and preprocess `series` argument in `ForecasterRecursiveMultiSeries`
  2453                                               class.
  2454                                           
  2455                                               Check if the indexes of the series are valid. If the temporal index of the
  2456                                               series is not a pandas DatetimeIndex index with frequency, it is overwritte

In [7]:
def funt_to_profile(series_dict):
    check_preprocess_series(series=series_dict)

%lprun -f check_preprocess_series funt_to_profile(series_dict)

Timer unit: 1e-07 s

Total time: 0.0202911 s
File: c:\Users\jaesc2\Miniconda3\envs\skforecast_py11_2\Lib\site-packages\skforecast\utils\utils.py
Function: check_preprocess_series at line 2448

Line #      Hits         Time  Per Hit   % Time  Line Contents
  2448                                           def check_preprocess_series(
  2449                                               series: pd.DataFrame | dict[str, pd.Series | pd.DataFrame],
  2450                                           ) -> tuple[dict[str, pd.Series], dict[str, pd.Index]]:
  2451                                               """
  2452                                               Check and preprocess `series` argument in `ForecasterRecursiveMultiSeries` class.
  2453                                           
  2454                                               - If `series` is a pandas DataFrame, it is converted to a dict of pandas 
  2455                                               Series and index is overwri

In [ ]:
# %%pyinstrument
# x_train, y_train = forecaster.create_train_X_y(
#     series = series_dict,
#     exog=exog_dict
# )

In [ ]:
import numpy as np
import pandas as pd
from numba import njit
from collections import defaultdict


n = 5000
series = pd.DataFrame(
    {i: np.random.randint(1, 100, n) for i in range(1, 100)
    },
    index=pd.date_range(start="2023-01-01", periods=n, freq="h"),
)

freq = series.index.freq
series.index.name = "datetime"
series = series.reset_index()
series = pd.melt(series, id_vars="datetime", var_name="series_id", value_name="value")
series = series.groupby("series_id").apply(
    lambda x: x.set_index("datetime").asfreq("D"), include_groups=False
)

In [ ]:


# Extract MultiIndex levels
ids = series.index.get_level_values(0).to_numpy()
dates = series.index.get_level_values(1).astype('datetime64[ns]').to_numpy()

# Ensure sorting by (id, datetime)
sort_idx = np.lexsort((dates, ids))
ids = ids[sort_idx]
dates_ns = dates[sort_idx].view('int64')

@njit
def most_common_deltas(ids, dates_ns, max_check=1000):
    result_ids = []
    result_deltas = []

    start = 0
    n = len(ids)

    while start < n:
        current_id = ids[start]
        end = start + 1
        while end < n and ids[end] == current_id:
            end += 1

        # Extract time series slice
        series = dates_ns[start:end]
        count = min(len(series) - 1, max_check)
        
        if count < 1:
            result_ids.append(current_id)
            result_deltas.append(-1)  # No data
            start = end
            continue

        delta_counts = {}
        for i in range(count):
            delta = series[i + 1] - series[i]
            if delta in delta_counts:
                delta_counts[delta] += 1
            else:
                delta_counts[delta] = 1

        # Find most common delta
        max_delta = -1
        max_count = -1
        for delta, cnt in delta_counts.items():
            if cnt > max_count:
                max_delta = delta
                max_count = cnt

        result_ids.append(current_id)
        result_deltas.append(max_delta)
        start = end

    return result_ids, result_deltas



In [ ]:

unique_ids, deltas_ns = most_common_deltas(ids, dates_ns)

# Convert to pandas timedelta64
freqs = {}
for uid, delta in zip(unique_ids, deltas_ns):
    if delta == -1:
        freqs[uid] = None  # Not enough data
    else:
        freqs[uid] = pd.Timedelta(delta, unit='ns')  # Convert to pandas Timedelta
freqs


In [13]:
freqstr_ = pd.date_range(start="2023-01-01", periods=10, freq='ME').freqstr

In [14]:
pd.date_range(start="2023-01-01", end="2023-01-10", freq=freqstr_)

DatetimeIndex([], dtype='datetime64[ns]', freq='ME')

In [12]:
s = pd.Series(np.arange(10), index=pd.date_range(start="2023-01-01", periods=10, freq='D'))
s

2023-01-01    0
2023-01-02    1
2023-01-03    2
2023-01-04    3
2023-01-05    4
2023-01-06    5
2023-01-07    6
2023-01-08    7
2023-01-09    8
2023-01-10    9
Freq: D, dtype: int64

In [13]:
s.loc['2021-01-01':'2024-01-05']

2023-01-01    0
2023-01-02    1
2023-01-03    2
2023-01-04    3
2023-01-05    4
2023-01-06    5
2023-01-07    6
2023-01-08    7
2023-01-09    8
2023-01-10    9
Freq: D, dtype: int64